In [16]:
import inseq
import pandas as pd
import warnings
import byt5_model
import torch

warnings.filterwarnings("ignore")

In [12]:
!pwd

/Users/auke/nlp-project-group-4


In [153]:
class InseqAttributer:
    def __init__(self, model="./models/test/", attribution_method="input_x_gradient") -> None:
        """Create an object that loads a given model through Inseq with a given attribution method"""
        self.model = inseq.load_model(model, attribution_method)
    
    def attribute(self, inp:str, out:str=None):
        """Use the Inseq model to generate feature attributions using a given input or input & output"""
        if inp and out:
            inp_out = (inp, out)
        else:
            inp_out = (inp,)

        inseq_out = self.model.attribute(*inp_out,
                                         attribute_target=True,
                                         step_scores=["probability"]
                                        )
        inseq_out.show()


def predict_on_data(data, model_config_path, spaces=True, device='auto'):
    if device == 'auto':
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    elif device == 'gpu' or device == 'cuda':
        device = torch.device("cuda")
    elif device == 'cpu':
        device = torch.device("cpu")
    else:
        print('Set device to auto, cpu, or gpu/cuda.')
        return


    if spaces:
        data['inputs'] = data['lemma'] + ' ' + data['features']
    else:
        data['inputs'] = data['lemma'] + data['features']


    data_gen_comparison = byt5_model.comparer(data, byt5_model.generate(model_config_path, data, device))
    data = pd.concat([data, data_gen_comparison], axis=1).rename(columns={'labels': 'plural'})

    correct = []
    for idx, row in data.iterrows():
        if row['Expected'] == row['Predicted']:
            correct.append('correct')
        else:
            correct.append('incorrect')

    data = data.assign(correct=correct)

    return (data[data.correct == 'incorrect'], data[data.correct == 'correct'])

def construct_character_spans(elements):
    spans = []
    for idx, char in enumerate(elements):
        # The capital A with accents denote the start of a two-bytes wide character, for turkish
#         These are the characters: ÇĞİÖŞÜçğıöşü
        if char.token in 'ÃÄÅ':
            spans.append((idx, idx+2))
    
    return spans


def inseq_on_predictions(samples, inseq_model, expected=False):
    # To only attribute from last vowel onward
    turkish_vowels = 'aeıioöuü'
    
    count = 0
    for idx, sample in samples.iterrows():
        count += 1
        if count > 1:
            print('\n')
        print('='*80)

        inp = sample['lemma'] + ' ' + sample['features']
        if expected:
            print('Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)')
            output_type = 'gold'
            out = sample['Expected']
        else:
            print('Testing Predicted (pred) outcome (what did the model take from the input during generation)')
            output_type = 'pred'
            out = sample['Predicted']

        print('Sample: {}\nIndex: {}\nThis prediction is {}\nInput -> output ({}): \'{}\' -> \'{}\''.format(
             count, idx, sample['correct'], output_type, inp, out
        ))
        if output_type == 'pred':
            print(f"The expected or gold output would be '{sample['Expected']}'")
            contrast = inseq_model.model.encode(sample['Expected'], as_targets=True)


        print('='*80)
        
        plural_suffix_start_index = max([-1] + [out.find(plural_suffix) for plural_suffix in ['ler', 'lar'] if plural_suffix in out])
#         print(f"{plural_suffix_start_index=}")

        # For the attribution start and end, the offset should be compute as byte-length, instead of utf-8 string length
#         plural_suffix_start_index = len(out[:plural_suffix_start_index].encode('utf-8'))
#         print(f"{out=}, {plural_suffix_start_index=}")

        # These suffixes are always 3 characters long ('ler' or 'lar')
        plural_suffix_end_index = plural_suffix_start_index + 3

        if plural_suffix_start_index > 0:  # The suffix is not always present in the output
            out = inseq_model.model.attribute(
                inp,
                out,
                show_progress=False,
                attr_pos_start=plural_suffix_start_index,  # These are unfortunately disabled, as they seem to interfere with the aggregation
#                 attr_pos_end=plural_suffix_end_index,
                attribute_target=True,
                step_scores=["probability"],
            )
        else:
            out = inseq_model.model.attribute(
                inp,
                out,
                show_progress=False,
                attribute_target=True,
                step_scores=["probability"],
            )
#         print(out.sequence_attributions[0])
        
        # Construct source and target spans
        source_spans = construct_character_spans(out.sequence_attributions[0].source) or None
        target_spans = construct_character_spans(out.sequence_attributions[0].target) or None
        print(source_spans, target_spans)
        
#         if source_spans or target_spans:
        if source_spans:
            out=inseq.data.aggregator.ContiguousSpanAggregator.aggregate(out.sequence_attributions[0], 
                                                                    source_spans=source_spans, 
                                                                    target_spans=target_spans)
        out.show()
            

## Turkish

Focus on relation between last vowel of the stem and the first vowel of the suffix. We expect the last vowel to be salient in predicting the first vowel of the suffix. We focus on plural nouns.

In [7]:
model_tur_finetuned = "./drive-symlink/NLP_project_morphological_inflection/finetuned_tur_1"
inseq_tur_finetuned = InseqAttributer(model_tur_finetuned, "input_x_gradient")

In [8]:
sample_runner(data_sampler('./data/tur.gold', model_tur_finetuned, device='cpu'), inseq_tur_finetuned, expected=True)

                      Expected                       Predicted
0        satıyor olmalı mıydık           satıyor olmalı mıydın
1                hamızlarından                  hamızlarınızda
2                  ovmaz mıydı          ovuyor olmamalı mıydın
3     yıpranmamalı mıymışsınız         yıpranmış olacak mıydık
4                   hüzünlerde                    hüzünlerinde
5                manivelanızda              manivelalarınızdan
6            dokuyor olmalısın            dokumuş olacak mıydı
7         barışmamalılar mıydı       barışıyor olmamalı mıydık
8      sahipli yazılımlarımızı             sahipli yazılımızda
9            yanıt vermedim mi           yanıt vermemeli miydi
10       onayacak olacak mısın                 onamamış mıydık
11          tıklamış olacaktın               tıklamamış mıydın
12              prostatınızdan                prostatlarınızda
13                damlarlar mı               damlamamış mıydık
14  hareket ediyor olacaktınız         hareket etmemeli

Attributing with input_x_gradient...: 100%|█████████████████████████████| 26/26 [00:21<00:00,  1.14it/s]


,a,t,Ä,±,l,m,Ä,±,Å,,,o,l,m,a,y,a,c,a,k,s,Ä,±,n,</s>
a,0.04,0.035,0.039,0.022,0.029,0.019,0.046,0.032,0.033,0.025,0.023,0.034,0.009,0.013,0.026,0.021,0.015,0.01,0.039,0.029,0.022,0.036,0.023,0.014,0.021
t,0.082,0.054,0.038,0.034,0.055,0.028,0.029,0.025,0.03,0.024,0.038,0.042,0.026,0.03,0.029,0.023,0.024,0.011,0.03,0.012,0.01,0.01,0.021,0.013,0.017
Ä,0.038,0.048,0.076,0.04,0.04,0.048,0.026,0.021,0.029,0.033,0.02,0.019,0.017,0.016,0.028,0.032,0.015,0.014,0.009,0.016,0.028,0.02,0.013,0.029,0.019
±,0.052,0.042,0.071,0.048,0.044,0.034,0.075,0.035,0.043,0.018,0.016,0.021,0.014,0.024,0.02,0.018,0.02,0.022,0.008,0.027,0.014,0.021,0.025,0.013,0.018
l,0.061,0.066,0.063,0.07,0.055,0.029,0.022,0.02,0.033,0.025,0.032,0.015,0.03,0.029,0.03,0.026,0.012,0.015,0.012,0.025,0.011,0.018,0.018,0.018,0.022
m,0.015,0.017,0.017,0.02,0.024,0.039,0.036,0.014,0.019,0.014,0.016,0.013,0.014,0.019,0.034,0.026,0.025,0.012,0.008,0.016,0.011,0.011,0.016,0.014,0.01
a,0.015,0.029,0.027,0.02,0.025,0.017,0.028,0.017,0.019,0.017,0.015,0.022,0.017,0.01,0.037,0.02,0.024,0.014,0.014,0.016,0.008,0.011,0.012,0.007,0.012
k,0.035,0.038,0.025,0.022,0.03,0.057,0.037,0.03,0.021,0.028,0.032,0.023,0.013,0.018,0.029,0.026,0.025,0.017,0.019,0.023,0.021,0.02,0.017,0.02,0.018
,0.057,0.043,0.031,0.024,0.036,0.035,0.017,0.031,0.017,0.016,0.033,0.01,0.014,0.01,0.025,0.045,0.014,0.027,0.011,0.02,0.017,0.017,0.017,0.011,0.01
V,0.013,0.012,0.013,0.009,0.015,0.012,0.01,0.008,0.008,0.007,0.011,0.007,0.007,0.009,0.012,0.011,0.006,0.006,0.008,0.012,0.009,0.008,0.01,0.007,0.01




Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 2
Index: 750
This prediction is incorrect
Input -> output (gold): 'katılmak V;OBLIG;SG;3;NEG;PRS;INTR' -> 'katılmameli mi'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 17/17 [00:10<00:00,  1.58it/s]


...

Attributing with input_x_gradient...: 100%|█████████████████████████████| 14/14 [00:06<00:00,  2.11it/s]


,i,l,k,e,l,e,r,i,n,d,e,n,</s>
i,0.073,0.1,0.145,0.167,0.069,0.035,0.079,0.081,0.07,0.141,0.032,0.033,0.145
l,0.197,0.082,0.092,0.059,0.104,0.077,0.071,0.065,0.037,0.066,0.028,0.089,0.038
k,0.051,0.077,0.063,0.044,0.056,0.06,0.074,0.089,0.021,0.045,0.076,0.029,0.088
e,0.121,0.058,0.132,0.148,0.072,0.024,0.03,0.037,0.069,0.026,0.035,0.075,0.058
,0.045,0.043,0.018,0.029,0.143,0.062,0.04,0.032,0.031,0.018,0.022,0.019,0.019
N,0.029,0.047,0.073,0.047,0.031,0.052,0.031,0.048,0.018,0.019,0.047,0.017,0.02
;,0.046,0.077,0.029,0.034,0.044,0.027,0.034,0.024,0.046,0.027,0.025,0.02,0.028
A,0.016,0.032,0.024,0.015,0.024,0.016,0.016,0.018,0.01,0.012,0.014,0.007,0.014
B,0.032,0.072,0.057,0.059,0.043,0.03,0.014,0.013,0.03,0.017,0.01,0.015,0.006
L,0.013,0.028,0.066,0.051,0.011,0.013,0.072,0.07,0.051,0.043,0.033,0.014,0.017




Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 10
Index: 1564
This prediction is correct
Input -> output (gold): 'seçilmek V;OBLIG;SG;1;NEG;PST;INTR;LGSPEC03' -> 'seçilmemeli miymişim'


Attributing with input_x_gradient...: 100%|█████████████████████████████| 24/24 [00:16<00:00,  1.36it/s]


,s,e,Ã,§,i,l,m,e,m,e,l,i,,m,i,y,m,i,Å,,i,m,</s>
s,0.023,0.05,0.055,0.052,0.023,0.022,0.017,0.018,0.01,0.01,0.008,0.016,0.021,0.012,0.009,0.01,0.01,0.005,0.009,0.024,0.005,0.012,0.011
e,0.026,0.073,0.039,0.031,0.025,0.03,0.044,0.014,0.015,0.01,0.007,0.012,0.02,0.016,0.005,0.015,0.014,0.006,0.017,0.01,0.007,0.01,0.008
Ã,0.019,0.033,0.145,0.036,0.037,0.048,0.018,0.021,0.015,0.014,0.02,0.031,0.031,0.019,0.01,0.012,0.034,0.017,0.014,0.017,0.015,0.022,0.031
§,0.017,0.047,0.046,0.069,0.069,0.057,0.029,0.041,0.026,0.026,0.022,0.021,0.022,0.024,0.022,0.028,0.019,0.017,0.027,0.012,0.013,0.021,0.02
i,0.021,0.027,0.033,0.05,0.053,0.028,0.031,0.023,0.02,0.019,0.029,0.008,0.008,0.02,0.01,0.023,0.009,0.006,0.009,0.006,0.014,0.025,0.009
l,0.014,0.015,0.069,0.019,0.074,0.041,0.039,0.015,0.018,0.013,0.018,0.012,0.009,0.01,0.007,0.009,0.019,0.007,0.02,0.022,0.019,0.026,0.007
m,0.007,0.013,0.021,0.022,0.025,0.024,0.063,0.038,0.036,0.031,0.019,0.013,0.014,0.014,0.018,0.013,0.009,0.012,0.013,0.008,0.014,0.017,0.016
e,0.007,0.019,0.015,0.018,0.021,0.021,0.014,0.028,0.024,0.022,0.021,0.017,0.014,0.018,0.012,0.011,0.008,0.006,0.01,0.008,0.009,0.011,0.006
k,0.019,0.033,0.05,0.058,0.052,0.036,0.027,0.059,0.028,0.037,0.026,0.02,0.022,0.026,0.032,0.03,0.033,0.018,0.022,0.012,0.016,0.025,0.024
,0.024,0.035,0.029,0.02,0.02,0.025,0.018,0.02,0.041,0.016,0.032,0.017,0.019,0.018,0.015,0.018,0.021,0.009,0.019,0.012,0.013,0.014,0.012


In [13]:
header_names = ['lemma', 'labels', 'features']

turkish_train = pd.read_csv('./data/tur_large.train', sep='\t', names=header_names)

In [14]:
turkish_train.head()

,lemma,labels,features
0,masör,masörlerdim,N;ARGNO1S;PL;PST
1,kışkırtmak,kışkırtmış mıyım,V;SG;1;POS;PST;INTR;LGSPEC03
2,küçük,küçük değildiler,ADJ;DECL;PL;3;NEG;PST;LGSPEC01
3,Yahudi,Yahudimizin,N;GEN;SG;PSS1P
4,bitirttirmek,bitirttirecek olacak mıymışım,V;PROSP;SG;1;POS;FUT;INTR;LGSPEC03


In [72]:
filtered_turkish = turkish_train[
    turkish_train['features'].str.startswith('N;')  # Ensure it is a noun
  & turkish_train['features'].str.contains('PL;')]  # And plural
print(filtered_turkish.size)
filtered_turkish.head()

1803


,lemma,labels,features
0,masör,masörlerdim,N;ARGNO1S;PL;PST
28,pipet,pipetlerimiz,N;NOM;PL;PSS1P
36,göt,götlerinin,N;GEN;PL;PSS3S
57,bozayı,bozayılarınızdan,N;ABL;PL;PSS2P
82,hırsız,hırsızlarımızı,N;ACC;PL;PSS1P


In [115]:
incorrect_turkish, correct_turkish = predict_on_data(filtered_turkish.head(40), model_tur_finetuned)

               Expected                Predicted
0           masörlerdim            masörlerinden
28         pipetlerimiz            pipetlerinden
36           götlerinin              götlerinden
57     bozayılarınızdan             bozayılar mı
82       hırsızlarımızı          hırsızlarınızda
92        durumlarından           durumlarınızda
96       şapkacılarında            şapkacılar mı
98        nöronlarından           nöronlarınızda
100        karınlarımız                karınızda
103       vurgularından           vurgularınızda
106     notiluslarımızı         notiluslarınızda
109          incilermiş             incilerinden
110       duraklarmışız              duraklar mı
127       bohçalarmışım              bohçalar mı
132   çığırtkanlarımızı          çığırtkanlar mı
144         atomlarının            atomlarınızda
147  ayrımcılıklardınız  ayrımcılık olacak mıydı
156  seri katillerimize       seri katillerinden
195         kirpilermiş               kirpilerdi
199         çöllerin

In [136]:
incorrect_attributions = inseq_on_predictions(incorrect_turkish, inseq_tur_finetuned, expected=False)

Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 1
Index: 0
This prediction is incorrect
Input -> output (pred): 'masör N;ARGNO1S;PL;PST' -> 'masörlerinden'
The expected or gold output would be 'masörlerdim'
plural_suffix_start_index=5
plural_suffix_start_index=6
GradientFeatureAttributionSequenceOutput({
    source: list with 24 elements of type TokenWithId,
    target: list with 15 elements of type TokenWithId:[
        'm', 'a', 's', 'Ã', '¶', 'r', 'l', 'e', 'r', 'i', 'n', 'd', 'e', 'n', '</s>'
    ],
    source_attributions: torch.float32 tensor of shape [24, 15, 1472] on cpu,
    target_attributions: torch.float32 tensor of shape [15, 15, 1472] on cpu,
    step_scores: {
        probability: torch.float32 tensor of shape [15] on cpu: [
             1.00,  1.00,  1.00,  0.99,  0.99,  1.00,  0.60,  0.62,  1.00,  0.37,  0.66,  0.86,  1.00,  0.60,  1.00
        ],
    },
    sequence_scores: None,
    attr_pos_start: 0,
    attr_pos_e

,m,a,s,Ã¶,r,l,e,r,i,n,d,e,n,</s>
m,0.105,0.083,0.076,0.075,0.026,0.023,0.037,0.031,0.098,0.037,0.037,0.03,0.043,0.057
a,0.071,0.113,0.103,0.052,0.055,0.038,0.059,0.074,0.067,0.042,0.039,0.019,0.03,0.043
s,0.052,0.042,0.043,0.03,0.056,0.026,0.034,0.029,0.023,0.018,0.016,0.02,0.031,0.012
Ã¶,0.069,0.155,0.122,0.114,0.117,0.066,0.034,0.168,0.072,0.047,0.033,0.074,0.043,0.092
r,0.035,0.092,0.148,0.117,0.106,0.078,0.018,0.079,0.078,0.024,0.039,0.114,0.041,0.035
,0.036,0.09,0.045,0.052,0.088,0.084,0.026,0.018,0.022,0.047,0.039,0.043,0.019,0.019
N,0.017,0.013,0.016,0.016,0.039,0.021,0.022,0.013,0.019,0.041,0.013,0.013,0.013,0.013
;,0.04,0.036,0.019,0.046,0.035,0.053,0.019,0.015,0.024,0.033,0.032,0.026,0.033,0.019
A,0.015,0.008,0.008,0.008,0.024,0.011,0.007,0.016,0.009,0.016,0.005,0.015,0.01,0.008
R,0.058,0.012,0.007,0.011,0.009,0.013,0.006,0.01,0.011,0.009,0.008,0.01,0.006,0.011




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 2
Index: 28
This prediction is incorrect
Input -> output (pred): 'pipet N;NOM;PL;PSS1P' -> 'pipetlerinden'
The expected or gold output would be 'pipetlerimiz'
plural_suffix_start_index=5
plural_suffix_start_index=5
GradientFeatureAttributionSequenceOutput({
    source: list with 21 elements of type TokenWithId,
    target: list with 14 elements of type TokenWithId:[
        'p', 'i', 'p', 'e', 't', 'l', 'e', 'r', 'i', 'n', 'd', 'e', 'n', '</s>'
    ],
    source_attributions: torch.float32 tensor of shape [21, 14, 1472] on cpu,
    target_attributions: torch.float32 tensor of shape [14, 14, 1472] on cpu,
    step_scores: {
        probability: torch.float32 tensor of shape [14] on cpu: [
             1.00,  1.00,  1.00,  0.99,  1.00,  0.85,  1.00,  1.00,  0.66,  0.52,  0.79,  0.99,  0.80,  1.00
        ],
    },
    sequence_scores: None,
    attr_pos_start: 0,
    attr_pos_end: 14,
})

,p,i,p,e,t,l,e,r,i,n,d,e,n,</s>
p,0.112,0.148,0.082,0.103,0.063,0.04,0.058,0.042,0.088,0.084,0.107,0.119,0.015,0.014
i,0.083,0.148,0.126,0.058,0.08,0.048,0.042,0.027,0.027,0.044,0.023,0.051,0.032,0.024
p,0.056,0.042,0.07,0.051,0.036,0.026,0.057,0.045,0.041,0.053,0.023,0.039,0.022,0.015
e,0.027,0.03,0.027,0.051,0.059,0.028,0.054,0.031,0.031,0.03,0.025,0.029,0.023,0.039
t,0.057,0.076,0.082,0.049,0.053,0.055,0.105,0.142,0.079,0.048,0.051,0.087,0.034,0.039
,0.088,0.038,0.048,0.08,0.062,0.137,0.016,0.031,0.044,0.036,0.017,0.015,0.016,0.015
N,0.018,0.055,0.043,0.021,0.025,0.037,0.035,0.036,0.02,0.027,0.036,0.014,0.043,0.02
;,0.047,0.035,0.023,0.024,0.047,0.024,0.021,0.045,0.027,0.039,0.026,0.028,0.024,0.023
N,0.028,0.016,0.017,0.013,0.015,0.01,0.008,0.01,0.012,0.012,0.01,0.011,0.008,0.013
O,0.056,0.015,0.016,0.014,0.022,0.041,0.022,0.019,0.025,0.015,0.017,0.015,0.015,0.022




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 3
Index: 36
This prediction is incorrect
Input -> output (pred): 'göt N;GEN;PL;PSS3S' -> 'götlerinden'
The expected or gold output would be 'götlerinin'
plural_suffix_start_index=3
plural_suffix_start_index=4
GradientFeatureAttributionSequenceOutput({
    source: list with 20 elements of type TokenWithId:[
        'g', 'Ã', '¶', 't', ' ', 'N', ';', 'G', 'E', 'N', ';', 'P', 'L', ';', 'P', 'S', 'S', '3', 'S', '</s>'
    ],
    target: list with 13 elements of type TokenWithId:[
        'g', 'Ã', '¶', 't', 'l', 'e', 'r', 'i', 'n', 'd', 'e', 'n', '</s>'
    ],
    source_attributions: torch.float32 tensor of shape [20, 13, 1472] on cpu,
    target_attributions: torch.float32 tensor of shape [13, 13, 1472] on cpu,
    step_scores: {
        probability: torch.float32 tensor of shape [13] on cpu: [
             1.00,  1.00,  1.00,  0.99,  0.52,  0.97,  1.00,  0.59,  0.77,  0.89,  1.00,  0.66

...

,k,o,r,s,a,n,l,a,r,Ä±,n,Ä±,z,d,a,</s>
k,0.075,0.137,0.098,0.142,0.125,0.098,0.055,0.056,0.044,0.106,0.084,0.068,0.09,0.093,0.03,0.061
o,0.069,0.055,0.054,0.041,0.057,0.049,0.058,0.043,0.055,0.063,0.04,0.038,0.037,0.049,0.029,0.019
r,0.168,0.034,0.055,0.162,0.104,0.133,0.062,0.048,0.026,0.035,0.015,0.031,0.026,0.049,0.016,0.018
s,0.03,0.082,0.131,0.121,0.138,0.127,0.104,0.075,0.03,0.109,0.057,0.081,0.058,0.116,0.019,0.038
a,0.02,0.026,0.024,0.032,0.029,0.04,0.027,0.067,0.083,0.037,0.037,0.102,0.022,0.031,0.114,0.026
n,0.165,0.191,0.225,0.036,0.046,0.033,0.07,0.139,0.062,0.065,0.075,0.049,0.028,0.038,0.054,0.191
,0.023,0.054,0.028,0.031,0.013,0.017,0.076,0.021,0.026,0.022,0.059,0.015,0.035,0.021,0.022,0.017
N,0.012,0.009,0.008,0.009,0.016,0.029,0.02,0.01,0.017,0.01,0.03,0.013,0.018,0.012,0.008,0.014
;,0.021,0.05,0.033,0.017,0.018,0.019,0.035,0.015,0.03,0.022,0.022,0.017,0.026,0.019,0.012,0.018
G,0.017,0.011,0.011,0.012,0.007,0.009,0.012,0.009,0.014,0.009,0.014,0.011,0.009,0.009,0.009,0.008




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 36
Index: 352
This prediction is incorrect
Input -> output (pred): 'müge N;ACC;PL;PSS1P' -> 'mügelerinden'
The expected or gold output would be 'mügelerimizi'
plural_suffix_start_index=4
plural_suffix_start_index=5
GradientFeatureAttributionSequenceOutput({
    source: list with 21 elements of type TokenWithId,
    target: list with 14 elements of type TokenWithId:[
        'm', 'Ã', '¼', 'g', 'e', 'l', 'e', 'r', 'i', 'n', 'd', 'e', 'n', '</s>'
    ],
    source_attributions: torch.float32 tensor of shape [21, 14, 1472] on cpu,
    target_attributions: torch.float32 tensor of shape [14, 14, 1472] on cpu,
    step_scores: {
        probability: torch.float32 tensor of shape [14] on cpu: [
             1.00,  1.00,  1.00,  1.00,  0.98,  0.94,  0.83,  1.00,  0.52,  0.73,  0.81,  1.00,  0.68,  1.00
        ],
    },
    sequence_scores: None,
    attr_pos_start: 0,
    attr_pos_end: 14,
})

,m,Ã¼,g,e,l,e,r,i,n,d,e,n,</s>
m,0.075,0.034,0.081,0.044,0.035,0.035,0.037,0.028,0.038,0.03,0.024,0.021,0.047
Ã¼,0.095,0.059,0.113,0.07,0.086,0.072,0.048,0.125,0.071,0.058,0.047,0.029,0.051
g,0.117,0.127,0.059,0.138,0.101,0.104,0.052,0.081,0.113,0.052,0.123,0.051,0.037
e,0.04,0.058,0.063,0.064,0.058,0.029,0.026,0.038,0.029,0.034,0.027,0.032,0.023
,0.149,0.072,0.048,0.068,0.13,0.041,0.026,0.027,0.046,0.021,0.025,0.013,0.027
N,0.092,0.036,0.03,0.014,0.022,0.012,0.034,0.025,0.021,0.039,0.011,0.024,0.015
;,0.026,0.062,0.062,0.023,0.053,0.019,0.033,0.03,0.03,0.037,0.04,0.02,0.035
A,0.027,0.011,0.039,0.014,0.014,0.01,0.01,0.021,0.011,0.009,0.009,0.01,0.01
C,0.072,0.042,0.032,0.087,0.078,0.041,0.045,0.043,0.039,0.014,0.048,0.037,0.058
C,0.042,0.052,0.038,0.04,0.017,0.033,0.02,0.053,0.038,0.033,0.012,0.027,0.046




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 37
Index: 355
This prediction is incorrect
Input -> output (pred): 'yanardağ N;DAT;PL;PSS1P' -> 'yanardağlarımız'
The expected or gold output would be 'yanardağlarımıza'
plural_suffix_start_index=8
plural_suffix_start_index=9
GradientFeatureAttributionSequenceOutput({
    source: list with 25 elements of type TokenWithId,
    target: list with 19 elements of type TokenWithId:[
        'y', 'a', 'n', 'a', 'r', 'd', 'a', 'Ä', '', 'l', 'a', 'r', 'Ä', '±', 'm', 'Ä', '±', 'z', '</s>'
    ],
    source_attributions: torch.float32 tensor of shape [25, 19, 1472] on cpu,
    target_attributions: torch.float32 tensor of shape [19, 19, 1472] on cpu,
    step_scores: {
        probability: torch.float32 tensor of shape [19] on cpu: [
             1.00,  1.00,  1.00,  1.00,  1.00,  1.00,  0.98,  1.00,  1.00,  0.60,  1.00,  1.00,  0.46,  1.00,  0.49,  0.99,  1.00,  1.00,  0.36
        ],
    },
   

,y,a,n,a,r,d,a,Ä,l,a,r,Ä±,m,Ä±,z,</s>
y,0.108,0.041,0.049,0.061,0.023,0.064,0.066,0.062,0.017,0.023,0.034,0.048,0.036,0.015,0.032,0.024
a,0.05,0.077,0.062,0.041,0.03,0.073,0.078,0.079,0.041,0.017,0.017,0.071,0.041,0.018,0.056,0.044
n,0.055,0.032,0.068,0.037,0.063,0.034,0.029,0.03,0.033,0.028,0.03,0.015,0.061,0.033,0.023,0.01
a,0.126,0.116,0.08,0.078,0.097,0.081,0.018,0.121,0.063,0.012,0.04,0.076,0.091,0.073,0.008,0.089
r,0.059,0.047,0.103,0.176,0.145,0.188,0.066,0.07,0.057,0.069,0.025,0.144,0.05,0.077,0.037,0.084
d,0.107,0.151,0.155,0.13,0.108,0.064,0.115,0.04,0.103,0.06,0.061,0.023,0.049,0.041,0.013,0.058
a,0.026,0.018,0.059,0.049,0.052,0.019,0.126,0.042,0.02,0.065,0.068,0.077,0.026,0.046,0.015,0.05
Ä,0.033,0.035,0.03,0.028,0.025,0.029,0.046,0.042,0.028,0.089,0.109,0.034,0.046,0.021,0.027,0.027
,0.014,0.025,0.01,0.011,0.024,0.049,0.024,0.069,0.078,0.063,0.038,0.032,0.022,0.018,0.072,0.017
N,0.014,0.013,0.01,0.008,0.009,0.013,0.007,0.014,0.018,0.013,0.017,0.01,0.032,0.012,0.044,0.012


In [154]:
correct_attributions = inseq_on_predictions(correct_turkish, inseq_tur_finetuned, expected=False)

Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 1
Index: 199
This prediction is correct
Input -> output (pred): 'çöl N;ABL;PL;PSS3S' -> 'çöllerinden'
The expected or gold output would be 'çöllerinden'
[(0, 2), (2, 4)] [(0, 2), (2, 4)]


,l,e,r,i,n,d,e,n,</s>
Ã§,0.126,0.093,0.104,0.086,0.063,0.053,0.046,0.045,0.037
Ã¶,0.23,0.06,0.059,0.06,0.061,0.032,0.052,0.031,0.035
l,0.113,0.164,0.138,0.174,0.039,0.027,0.052,0.032,0.039
,0.03,0.076,0.034,0.033,0.032,0.017,0.03,0.045,0.03
N,0.025,0.02,0.027,0.013,0.018,0.023,0.023,0.02,0.016
;,0.064,0.026,0.047,0.022,0.04,0.032,0.036,0.031,0.042
A,0.01,0.012,0.01,0.009,0.01,0.007,0.008,0.012,0.008
B,0.043,0.015,0.013,0.018,0.02,0.021,0.018,0.011,0.011
L,0.016,0.045,0.041,0.01,0.038,0.009,0.017,0.015,0.021
;,0.016,0.016,0.026,0.018,0.025,0.013,0.022,0.019,0.014




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 2
Index: 203
This prediction is correct
Input -> output (pred): 'malumat N;LOC;PL;PSS2P' -> 'malumatlarınızda'
The expected or gold output would be 'malumatlarınızda'
None [(10, 12), (13, 15)]


,l,a,r,Ä,±,n,Ä,±,z,d,a,</s>
m,0.035,0.035,0.031,0.051,0.017,0.036,0.026,0.038,0.024,0.031,0.036,0.084
a,0.03,0.013,0.015,0.044,0.06,0.025,0.014,0.029,0.019,0.043,0.012,0.044
l,0.055,0.169,0.131,0.04,0.061,0.051,0.139,0.073,0.051,0.069,0.059,0.065
u,0.027,0.017,0.02,0.027,0.047,0.035,0.026,0.033,0.033,0.026,0.024,0.025
m,0.057,0.034,0.046,0.054,0.049,0.074,0.03,0.038,0.047,0.053,0.074,0.045
a,0.161,0.075,0.102,0.149,0.075,0.068,0.094,0.08,0.115,0.112,0.097,0.089
t,0.028,0.044,0.022,0.029,0.037,0.039,0.052,0.075,0.082,0.048,0.072,0.03
,0.053,0.026,0.014,0.038,0.016,0.013,0.032,0.01,0.018,0.025,0.02,0.027
N,0.011,0.014,0.01,0.008,0.012,0.014,0.014,0.02,0.01,0.01,0.018,0.011
;,0.016,0.015,0.015,0.022,0.019,0.019,0.014,0.019,0.02,0.016,0.016,0.019




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 3
Index: 329
This prediction is correct
Input -> output (pred): 'elektrikli sandalye N;ABL;PL;PSS2S' -> 'elektrikli sandalyelerinden'
The expected or gold output would be 'elektrikli sandalyelerinden'
None None


,l,e,r,i,n,d,e,n,</s>
e,0.017,0.019,0.068,0.009,0.022,0.011,0.011,0.024,0.024
l,0.033,0.012,0.037,0.014,0.057,0.008,0.029,0.046,0.018
e,0.084,0.014,0.062,0.045,0.021,0.006,0.061,0.009,0.014
k,0.023,0.02,0.027,0.014,0.015,0.008,0.012,0.014,0.015
t,0.039,0.013,0.012,0.008,0.011,0.018,0.019,0.015,0.017
r,0.016,0.037,0.021,0.018,0.019,0.008,0.022,0.021,0.006
i,0.009,0.019,0.011,0.007,0.013,0.004,0.009,0.004,0.011
k,0.012,0.022,0.018,0.014,0.012,0.012,0.015,0.009,0.013
l,0.015,0.043,0.01,0.016,0.036,0.008,0.02,0.019,0.006
i,0.023,0.034,0.011,0.032,0.011,0.019,0.011,0.013,0.01
